In [ ]:
%run calccomp.ipynb

In [ ]:
%run gammaFV3.ipynb

In [ ]:
%run gammaNV3.ipynb

In [ ]:
%run gammaIP.ipynb

In [ ]:
%run Kthermo.ipynb

In [ ]:
%run Kcond.ipynb

In [ ]:
%run alphaV3.ipynb

In [ ]:
%run speciation.ipynb

In [ ]:
%run gammaTV3.ipynb

In [ ]:
# SHI_AMI      SHI_AMI calculates speciation for a number of elements in seawater 
#              given sufficient input:
#                
#              
#              INPUT:
#              PAR1  (some unit) : scalar or vector of size n x 1
#              PAR2  (some unit) : scalar or vector of size n x 1
#              PAR1TYPE       () : scalar or vector of size n x 1(*)
#              PAR2TYPE       () : scalar or vector of size n x 1(*)
#              () : scalar or vector of size n x 1
#              Tc      (degr. C) : scalar or vector of size n x 1
#              P         ( dbar) : scalar or vector of size n x 1
#              SIL    (umol/kgSW): scalar or vector of size n x 1
#              PO4    (umol/kgSW): scalar or vector of size n x 1
#              pHSCALE           : scalar or vector of size n x 1(**)
#
#             (*) Each element must be an integer, 
#              indicating that PAR1 (or PAR2) is of type: 
#              1 = Total Alkalinity
#              2 = DIC
#              3 = pH
#              4 = pCO2
#              5 = fCO2
# 
#             (**) Each element must be an integer, 
#              indicating that the pH-input (PAR1 or PAR2, if any) is at:
#              1 = Total scale
#              2 = Seawater scale
#              3 = Free scale
#              4 = NBS scale
#
##              NUMBERING SYSTEM
#               Major Cations    
#               Na = 0;       K = 1;     Mg = 2;       Ca = 3;      Sr = 4
#
#               ************************************************************************************
#
#               Minor Cations
#                H = 5;      Li = 6;      Rb = 7;       Cs = 8;      NH4 = 9;    Ba = 10;   Mn = 11;    
#           Fe(II)= 12;     Co = 13;     Ni = 14;   Cu(II)= 15;      Zn = 16;   UO2 = 17;   Be = 18;    
#              Cd = 19;  Pb(II)= 20;   Cu(I)= 21;      La = 22;      Ce = 23;    Pr = 24;   Nd = 25;      
#              Pm = 26;     Sm = 27;     Eu = 28;      Gd = 29;      Tb = 30;    Dy = 31;   Ho = 32;      
#              Er = 33;     Tm = 34;     Yb = 35;      Lu = 36;       Y = 37;    Al = 38;   Ga = 39;      
#              In = 40; Fe(III)= 41;
#
#               ************************************************************************************
#
#               Major Anions   
#               Cl = 0;     SO4 = 1;     CO3 = 2;     HCO3 = 3;    Br = 4;     F = 5   B(OH)4 = 6
# 
#               ************************************************************************************
#
#              Minor Anions
#                  OH = 7;   HSO4 = 8;     HS = 9;      I = 10;    ClO3  = 11;  ClO4 = 12;    BrO3 = 13;     
#                CNS = 14;   NO2 = 15;   NO3 = 16;  H2PO4 = 17;     HPO4 = 18;   PO4 = 19;  H2AsO4 = 20;  
#              HAsO4 = 21;  AsO4 = 22;  HSO3 = 23;    SO3 = 24;  Acetate = 25;
#          
#               ************************************************************************************
#   

##  Assign values to global variables
mjCaMax = 5
mnCaMax = 37
  CaMax = mjCaMax + mnCaMax

mjAnMax = 7
mnAnMax = 19 
  AnMax = mjAnMax + mnAnMax

NeutMax = 7

def SHI_AMI(S, Tc, P, PAR1, PAR2, PAR1TYPE, PAR2TYPE, SIL, PO4, pHSCALE):
    
## Intiating Speciation calculation:
# INPUT: S, Tc, P, PAR1, PAR2, PAR1TYPE, PAR2TYPE, SIL, PO4, pHSCALE
## --------------------------------------------------------------------------------------------

## 1) Ionic Strength and composition of background electrolyte 
    I, mT_cat, mT_an, OH = calccomp (S, Tc, P, PAR1, PAR2, PAR1TYPE, PAR2TYPE, SIL, PO4, pHSCALE) 

## --------------------------------------------------------------------------------------------

## 2) Free Activity coefficients [gammaV3.m; gammaNV3.m]
# Like Millero Model, we calculate gamma using Pitzer equations for :
#
# Major Cations                  Minor Cations 
# Na, K, Mg, Ca, Sr              H, Li, Rb, Cs, NH4, Ba, Mn, Fe(II), Co, Ni, Cu(II), Zn, UO2,
#                                Be, Cd, Pb(II), Cu(I)
#                               
#                                REE
#                                La, Ce, Pr, Nd, Pm, Sm, Eu, Gd, Tb, Dy, Ho, Er, Tm, Yb, Lu, Y
#                                
#                                Trivalent
#                                Al, Ga, In, Fe(III)
#
#
# Major Anions                   Minor Anion
# Cl, SO4, CO3, HCO3             OH, HSO4, HS,  I, ClO3, ClO4, BrO3, CNS, NO2, NO3, H2PO4, HPO4,
# Br, F, B(OH)4                  PO4, H2AsO4, HAsO4, AsO4, HSO3, SO3, Acetate
#
# Neutral:
# NH3, B(OH)3, H3PO4, H2S, SO2, HF, CO2
#
# Charged ion pairs whose pitzer parameters are available
# MgOH*  MgF*  CaF*
#
# Neutral ion pairs and charged ion pairs for which no pitzer data is available: 
# For a number of ion pairs, we have assigned assumed activity coefficients.
# For detailed description of assumptions made, see "gammaIP.m" or type "help gammaIP" 
# in the command window.
#
# Pitzer parameters that are needed  are generated by (and can be edited in) CreatePitzerV3.m
# ****************************************************************************************************************************
    gF_cat, gF_an, gIP1, gdivIP, gtrivIP = gammaFV3(Tc, I, mT_cat,mT_an)
    gN = gammaNV3(Tc,I,mT_cat,mT_an)
    gmjIP, mnCatIP, gdivIP, gtrivIP = gammaIP(Tc,I, mT_cat, mT_an)
# ****************************************************************************************************************************

## --------------------------------------------------------------------------------------------------------------
## 3) Thermodynamic/conditional association constants
# Equilbrium constants used in the chemical equilibria can be:
#
# Thermodynamic constants K:                - expressed in terms of activity 
#                                           - Functions of temperature and pressure only. 
#
# Stoichiometric/conditional constants K?:  - expressed in terms of concentrations 
#                                           - functions of temperature, pressure and 
#                                             solution composition. 
#
# "This dependence on solution compositionvia the activity coefficients limits the 
# practical value  of stoichiometric constants.  Values of the stoichiometric constant 
# K* measured in  normal seawater can be used only for solutions of seawater composition 
# and for the salinity, or salinities, for which K* has been determined.  Any variations 
# from seawater compositionsuch as might be found in enclosed seas ,pore waters, 
# and some estuarieswill lead to changes  in the value of ... activity coefficients 
# in the  equation ... and therefore a change in K*" (TURNER ET AL 2016)
#
# In SHIAMI: 
# 1) Thermodynamic log Ks are compiled from various sources. SEE "Kthermo.m" and 
#    "thermodatabase1.m"
# 
# 2) Desired stoichiometric constants are calculated from thermodynamic log Ks using the 
#    model calculated free activity coefficients.
# 
# ****************************************************************************************************************************
    mjKthermo, mnCat_Kthermo, divKthermo, trivKthermo, mnAn_Kthermo = Kthermo(Tc)
    mjKcond, mnCat_Kcond, divKcond, trivKcond  = Kcond(Tc, I, mT_cat,mT_an)
# ****************************************************************************************************************************

## ###########################################################################################################
## CALCULATING SPECIATION: 
# Using the parameters listed above, one can determine the distribution of elements 
# in an aqueous solution  via the following means:

## --------------------------------------------------------------------------------------------------------------
## 1) Calculating "alpha" or the fraction of free ions in a solution of fixed composition [alphaV3.m] 
# To calculate the percentage/fraction of free cation or anion in the solution, we
# use the "ion association/ion-pairing model. We solve a series of linear
# equations through an iterative method which is initiated by setting the
# alpha_cat = 1 i.e [M]F = [M]T. 

# ****************************************************************************************************************************
    alpha_cat, alpha_an, mF_an, mF_cat, itera = alphasV3(Tc, I, mT_cat,mT_an,OH)
# ****************************************************************************************************************************

## --------------------------------------------------------------------------------------------------------------
## 2) Calculating speciation
# Using alpha determined above, it is possible to determine the speciation:
# ****************************************************************************************************************************
    mjcat_spec, mjcat_FREE, mjan_spec, mjan_FREE, mncat_spec, mncat_FREE, div_spec, div_FREE, triv_spec, triv_FREE = speciation (Tc, I, mT_cat, mT_an, OH)
# ****************************************************************************************************************************

## ###########################################################################################################
## Correcting activity coefficients for complex formation: 
# From Hain et al 2015:
# 'There are two reasons why the activity of a given chemical species (e.g., CO32-) is different from its
# stoichiometric concentration: 

# (1) The electrostatic ion-ion interactions in electrolyte solutions typically reduce the activity 
#     of a given species below its free concentration Debye and Hückel, 1923; Pitzer, 1991). 
#     [This is accounted for by free activity coeefficients]

# (2) Some ions form strong bonds with each other (i.e., complex formation or ion pairing;
#     Garrels and Thompson, 1962) such that only a fraction of that species can be said to be free'
#     [This is accounted for by the desired total activity coeefficients.]

## Calculating Total Activity coefficents [gammaTV3.m]
# ai = mf * yf               (1)
# mf = mT * alpha            (2)
# ai = mT * alpha * yF       (3) = (2) -> (1)
# yT = alpha * yF            (4)
# ai = mT * yT               (5) = (4) -> (3)

# When the ion does not form an ion pair or alpha = 1.0 the values of yT = yF. 
# If the value of yF is determined for a solution (e.g. Na-Mg-C1-SO4) using the
# Pitzer equations, the ion pairing model can be used to account for strong
# interactions between i (Cu 2+) and minor anions (CO~-):

# Using the alpha determined via iteration,  we can convert yF to yT: 

# ****************************************************************************************************************************
    gT_cat, gT_an = gammaTV3 (alpha_an, alpha_cat, gF_an, gF_cat)
# ****************************************************************************************************************************

    return gT_cat, gT_an, gF_cat, gF_an, gN, gmjIP, mnCatIP, gdivIP, gtrivIP, mT_cat, mT_an, mF_an, mF_cat, alpha_an, alpha_cat, mjcat_spec, mjcat_FREE, mjan_spec, mjan_FREE, mncat_spec, mncat_FREE, div_spec, div_FREE, triv_spec, triv_FREE 